## 1. 数据加载


In [1]:
import pandas as pd

In [2]:
# 加载 FB15k-237 数据集的训练、验证和测试集
data_dir = "./knowledge_graph_completion/data/OpenBG500/"
train_df, valid_df = pd.read_csv(data_dir+"train.csv"),pd.read_csv(data_dir+"test.csv")
print(f"训练集大小: {len(train_df)} 条三元组")
print(f"验证集大小: {len(valid_df)} 条三元组")


训练集大小: 8707 条三元组
验证集大小: 1187 条三元组


In [3]:
train_df[['head','relation','tail']].head()

,head,relation,tail
0,婴儿围嘴,文胸尺码,16*10CM*2个
1,大码内衣女超薄款大胸显小神器缩胸防下垂文胸胖mm200斤胸罩夏季,文胸尺码,如担心下围短，可联系客服送排扣
2,卫衣,文胸尺码,均
3,卫衣,文胸尺码,不加绒
4,诺瓦纳内衣,文胸尺码,S下胸围65-80


## 2. 数据预览

In [4]:
# 定义一个函数便于打印三元组列表的前几项
def preview_triples(name, df, num=5):
    print(f"\n{name} 集合样本前 {num} 条：")
    for _, row in df.head(num).iterrows():
        print(f"{row['head']}\t{row['relation']}\t{row['tail']}")
    print("...")

In [5]:
preview_triples("训练", train_df, num=5)
preview_triples("验证", valid_df, num=5)
# preview_triples("测试", test_df, num=5)


训练 集合样本前 5 条：
婴儿围嘴	文胸尺码	16*10CM*2个
大码内衣女超薄款大胸显小神器缩胸防下垂文胸胖mm200斤胸罩夏季	文胸尺码	如担心下围短，可联系客服送排扣
卫衣	文胸尺码	均
卫衣	文胸尺码	不加绒
诺瓦纳内衣	文胸尺码	S下胸围65-80
...

验证 集合样本前 5 条：
文胸	文胸尺码	常规
超薄运动内衣大胸健身减震跑步大码运动文胸200斤防震女胖mm夏季	文胸尺码	本款文胸偏大，建议咨询客服
卫衣	文胸尺码	不加绒
少女文胸	文胸尺码	均码
文胸	文胸尺码	常规
...


## 3. 构建 Few-Shot 提示

In [6]:
triples = list(train_df[['head','relation','tail']].itertuples(index=False, name=None))


In [7]:
# 从训练集中选择关系为 "founded" 的 few-shot 示例
relation = "文胸尺码"
K = 3  # 选取3条few-shot示例
fewshot_examples = [trip for trip in triples if trip[1] == relation][:K]

In [8]:
fewshot_examples

[('婴儿围嘴', '文胸尺码', '16*10CM*2个'),
 ('大码内衣女超薄款大胸显小神器缩胸防下垂文胸胖mm200斤胸罩夏季', '文胸尺码', '如担心下围短，可联系客服送排扣'),
 ('卫衣', '文胸尺码', '均')]

In [9]:
query_head = "卫衣"
query_relation = "文胸尺码"
prompt = ""
for h, r, t in fewshot_examples:
    prompt += f"{h} - {r} -> {t}\n"
prompt += f"{query_head} - {query_relation} ->"

print("\n构建的提示 Prompt:\n" + prompt)


构建的提示 Prompt:
婴儿围嘴 - 文胸尺码 -> 16*10CM*2个
大码内衣女超薄款大胸显小神器缩胸防下垂文胸胖mm200斤胸罩夏季 - 文胸尺码 -> 如担心下围短，可联系客服送排扣
卫衣 - 文胸尺码 -> 均
卫衣 - 文胸尺码 ->


## 4. 模型推理生成尾实体

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/root/anaconda3/envs/kgc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# 加载 Qwen3-0.6B 模型和分词器
model_name = "./knowledge_graph_completion/models/Qwen3-0.6B/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [13]:
model.to(device)                      # 把模型权重移动到 GPU
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [14]:
# 将提示编码为模型输入并生成文本
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs.input_ids.to(device)            # GPU Tensor
attention_mask = inputs.attention_mask.to(device)  # 如果有

In [15]:
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=10,
    do_sample=False
)

/root/anaconda3/envs/kgc/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/anaconda3/envs/kgc/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/root/anaconda3/envs/kgc/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [16]:
#    只取 prompt 后生成部分
generated_ids = outputs[0, input_ids.shape[1]:].cpu()
raw = tokenizer.decode(generated_ids, skip_special_tokens=True)

In [17]:
# 只保留第一行（遇到换行就截断）
pred = raw.splitlines()[0].strip()
print("预测尾实体：", pred)

预测尾实体： 16*10CM*2个


## 5. Hits@1 和 MRR 指标评估

In [18]:
from tqdm import tqdm

# 评估前 N 条测试集样本
N = min(2000, len(valid_df))
hits1_count = 0
ranks = []

In [19]:
for i, (_, row) in enumerate(tqdm(valid_df.head(N).iterrows(), total=N, desc="Evaluating")):
    
    # 直接从 row 里取列：
    h = row['head']      # 或者 row['head']，如果想用 MID
    r = row['relation']       # 原始谓词标识
    true_t = row['tail'] # 或者 row['tail']
    
    # 构建 Few-Shot 提示
    fs = train_df[train_df['relation'] == r].head(3)
    prompt_i = ""
    for _, ex in fs.iterrows():
        prompt_i += f"{ex['head']} - {ex['relation']} -> {ex['tail']}\n"
    prompt_i += f"{h} - {r} ->"
    
    # 模型生成预测
    inputs_i = tokenizer(prompt_i, return_tensors="pt")
    input_ids = inputs_i.input_ids.to(device)            # GPU Tensor
    attention_mask = inputs_i.attention_mask.to(device)  # 如果有
    outputs = model.generate(input_ids=input_ids,attention_mask=attention_mask,max_new_tokens=10,do_sample=False)
    generated_ids = outputs[0, input_ids.shape[1]:].cpu()
    pred = tokenizer.decode(generated_ids, skip_special_tokens=True).splitlines()[0].strip()

    # 统计 Hits@1 与 MRR
    hit = int(pred == true_t)
    hits1_count += hit
    ranks.append(1.0 if hit else 0.0)

Evaluating: 100%|██████████| 1187/1187 [14:26<00:00,  1.37it/s] 


In [34]:
hits1 = hits1_count / N
mrr   = sum(ranks) / N
print(f"Hits@1 = {hits1:.4f}, MRR = {mrr:.4f}")

Hits@1 = 0.4170, MRR = 0.4170


## 6. 实际测试

In [12]:
from knowledge_graph_completion.infer import evaluate_model

In [13]:
hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=0)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

Evaluating: 100%|██████████| 1187/1187 [06:07<00:00,  3.23it/s]

Hits@1: 0.0219, Hits@3: 0.0514, Hits@10: 0.0708, MRR: 0.0396


In [14]:
#N = min(100, len(valid_df))

hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=1)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")


Evaluating: 100%|██████████| 1187/1187 [06:31<00:00,  3.03it/s]

Hits@1: 0.3555, Hits@3: 0.4078, Hits@10: 0.4414, MRR: 0.3855


In [15]:
hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=3)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

Evaluating: 100%|██████████| 1187/1187 [07:37<00:00,  2.60it/s]

Hits@1: 0.4204, Hits@3: 0.5063, Hits@10: 0.5569, MRR: 0.4688


In [16]:
hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=5)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

Evaluating: 100%|██████████| 1187/1187 [12:36<00:00,  1.57it/s]

Hits@1: 0.4642, Hits@3: 0.5619, Hits@10: 0.6192, MRR: 0.5194


### 6.1 fewshot不同数量对比

In [21]:
from tqdm import tqdm

In [23]:
for i in range(2,15,2):
    print("*"*100)
    print("num_examples=",i)
    hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=i)
    print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

****************************************************************************************************
num_examples= 7


Evaluating: 100%|██████████| 1187/1187 [17:46<00:00,  1.11it/s]


Hits@1: 0.4836, Hits@3: 0.5737, Hits@10: 0.6361, MRR: 0.5357
****************************************************************************************************
num_examples= 9


Evaluating: 100%|██████████| 1187/1187 [16:41<00:00,  1.18it/s]


Hits@1: 0.5131, Hits@3: 0.5998, Hits@10: 0.6757, MRR: 0.5664
****************************************************************************************************
num_examples= 11


Evaluating: 100%|██████████| 1187/1187 [23:01<00:00,  1.16s/it]


Hits@1: 0.5055, Hits@3: 0.6226, Hits@10: 0.6900, MRR: 0.5715
****************************************************************************************************
num_examples= 13


Evaluating: 100%|██████████| 1187/1187 [19:11<00:00,  1.03it/s]

Hits@1: 0.5324, Hits@3: 0.6487, Hits@10: 0.7094, MRR: 0.5960


In [25]:
for i in range(2,15,2):
    print("*"*100)
    print("num_examples=",i)
    hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, num_examples=i)
    print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

****************************************************************************************************
num_examples= 2


Evaluating: 100%|██████████| 1187/1187 [07:04<00:00,  2.80it/s]


Hits@1: 0.3934, Hits@3: 0.4541, Hits@10: 0.5046, MRR: 0.4287
****************************************************************************************************
num_examples= 4


Evaluating: 100%|██████████| 1187/1187 [08:10<00:00,  2.42it/s]


Hits@1: 0.4524, Hits@3: 0.5383, Hits@10: 0.5855, MRR: 0.5005
****************************************************************************************************
num_examples= 6


...

Hits@1: 0.5173, Hits@3: 0.6268, Hits@10: 0.6799, MRR: 0.5752
****************************************************************************************************
num_examples= 12


Evaluating: 100%|██████████| 1187/1187 [12:41<00:00,  1.56it/s]


Hits@1: 0.5257, Hits@3: 0.6293, Hits@10: 0.7009, MRR: 0.5853
****************************************************************************************************
num_examples= 14


Evaluating: 100%|██████████| 1187/1187 [13:51<00:00,  1.43it/s]

Hits@1: 0.5307, Hits@3: 0.6268, Hits@10: 0.6891, MRR: 0.5845


In [15]:
print(f"Hits@1 = {hits1:.4f}, MRR = {mrr:.4f}")

Hits@1 = 0.3110, MRR = 0.3441


## 7. LoRA训练

In [1]:
from knowledge_graph_completion.infer import evaluate_model
from knowledge_graph_completion.utils import calculate_metrics
from knowledge_graph_completion.data.data_loader import load_openbg500
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/root/anaconda3/envs/kgc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 加载数据（已含 head_name, tail_name, relation_name）
import pandas as pd
data_dir = "./knowledge_graph_completion/data/OpenBG500/"
train_df, valid_df = pd.read_csv(data_dir+"train.csv"),pd.read_csv(data_dir+"test.csv")

In [4]:
train_df.head()

,head,relation,tail
0,婴儿围嘴,文胸尺码,16*10CM*2个
1,大码内衣女超薄款大胸显小神器缩胸防下垂文胸胖mm200斤胸罩夏季,文胸尺码,如担心下围短，可联系客服送排扣
2,卫衣,文胸尺码,均
3,卫衣,文胸尺码,不加绒
4,诺瓦纳内衣,文胸尺码,S下胸围65-80


In [5]:
# 加载分词器与模型
model_name = "./knowledge_graph_completion/models/Qwen3-0.6B/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

### 训练

In [7]:
from knowledge_graph_completion.train import train_model

In [8]:
# 2. 构建微调样本（随机挑 1000 条）
sample_df_train = train_df.sample(n=1000, random_state=42).reset_index(drop=True)
sample_df_test = valid_df.sample(n=1000, random_state=42).reset_index(drop=True)

In [9]:
sample_df_test

,head,relation,tail
0,相伴一生办公桌垫 超大号可定制尺寸图案鼠标垫电脑书桌垫子男女笔记本键盘垫家用桌垫学生 写字桌...,单面双面,双面
1,紫砂杯,保温时长,6小时以下
2,有机大米,是否转基因,否
3,婴儿电热蚊香液,是否含香味,无香
4,俄罗斯巧克力,糖种类,喜糖
...,...,...,...
995,【下单立减450元】realme 真我GT 5G高通骁龙888处理器65W闪充游戏学生手机,CPU型号,骁龙888
996,熟冻智利帝王蟹,保鲜工艺,熟冻
997,带鱼段,保鲜工艺,常温
998,婴儿床垫,床垫类型,乳胶床垫


In [10]:
# train_model 内部会将验证集上的最佳模型保存在 outputs/best_model/
train_model(
    model=model,
    tokenizer=tokenizer,
    train_data=sample_df_train,      # DataFrame，含 head/relation/tail 和 head_name/... 列
    valid_data=sample_df_test,      # DataFrame，用于早停 & 模型选择
    epochs=3,
    batch_size=16,
    learning_rate=5e-5,
    output_dir="./outputs/openBG500/",     # best_model 将保存在 outputs/best_model/
    num_examples=3,
    top_k=10
)
print("微调训练完毕，最佳模型已保存到 outputs/best_model/")

Epoch 1:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 63/63 [02:32<00:00,  2.43s/it]


[Epoch 1] 平均 Loss: 0.8976


Evaluating: 100%|██████████| 1000/1000 [06:26<00:00,  2.59it/s]


Hits@1: 0.1790, Hits@3: 0.2380, Hits@10: 0.2760, MRR: 0.2127
→ 保存最佳模型，MRR: 0.2127


...

Hits@1: 0.1660, Hits@3: 0.2100, Hits@10: 0.2670, MRR: 0.1955


Epoch 3: 100%|██████████| 63/63 [02:43<00:00,  2.60s/it]


[Epoch 3] 平均 Loss: 0.4245


Evaluating: 100%|██████████| 1000/1000 [06:24<00:00,  2.60it/s]


Hits@1: 0.1830, Hits@3: 0.2500, Hits@10: 0.3050, MRR: 0.2233
→ 保存最佳模型，MRR: 0.2233
训练完成，最佳模型保存在: ./outputs/openBG500/best_model
微调训练完毕，最佳模型已保存到 outputs/best_model/


### 测试集

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [27]:
adapter_dir      = "/root/szl/KG/knowledge_graph_completion/outputs/checkpoint-3450"
base_model_name  = "./knowledge_graph_completion/models/Qwen3-0.6B/"

# 1. 先加载基础模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name)

In [28]:
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [29]:
# 加载数据（已含 head_name, tail_name, relation_name）
import pandas as pd
data_dir = "./knowledge_graph_completion/data/OpenBG500/"
train_df, valid_df = pd.read_csv(data_dir+"train.csv"),pd.read_csv(data_dir+"test.csv")

In [30]:
valid_df.head(5)

,head,relation,tail
0,文胸,文胸尺码,常规
1,超薄运动内衣大胸健身减震跑步大码运动文胸200斤防震女胖mm夏季,文胸尺码,本款文胸偏大，建议咨询客服
2,卫衣,文胸尺码,不加绒
3,少女文胸,文胸尺码,均码
4,文胸,文胸尺码,常规


In [31]:
from knowledge_graph_completion.infer import evaluate_model

In [32]:
hits1, hits3, hits10, mrr = evaluate_model(model, tokenizer, valid_df, train_df, fewshot_k=3)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

Evaluating: 100%|██████████| 1187/1187 [37:48<00:00,  1.91s/it]

Hits@1: 0.4204, Hits@3: 0.5063, Hits@10: 0.5569, MRR: 0.4688


In [11]:
# 2. 用 PeftModel 把 adapter 权重叠加上去
peft_model = PeftModel.from_pretrained(model, adapter_dir)
# 将模型移动至设备
peft_model = peft_model.to(device)
peft_model.eval()

/root/anaconda3/envs/kgc/lib/python3.11/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features=1024, out_feat

In [12]:
hits1, hits3, hits10, mrr = evaluate_model(peft_model, tokenizer, valid_df, train_df, fewshot_k=3)
print(f"Hits@1: {hits1:.4f}, Hits@3: {hits3:.4f}, Hits@10: {hits10:.4f}, MRR: {mrr:.4f}")

Evaluating: 100%|██████████| 1187/1187 [09:41<00:00,  2.04it/s]

Hits@1: 0.4516, Hits@3: 0.5350, Hits@10: 0.5948, MRR: 0.4979
